In [8]:
import pandas as pd
import numpy as np
import utilities as ut

database=pd.read_csv("database_corregido.csv",encoding='UTF-8')

In [3]:
database.head()

,user_id,user_name,usersurname,email,password,year_birth,gender,mother_tongue,years_in,studies,working,support_type,expert,area,about_me,date,country,pic
0,1,ZpafJwb,IGLxqLcD,SidQXyFROW@gmail.com,FuDgmKsOA,1997,Femenino,"Frances, Chino",De 1 año a 2 años,Sin graduado escolar,Si,Emocional,No,Valle del Jarama,"Juegos, Baile, Musica",13/10/2021,NaN,NaN
1,2,vraVenl,pQgDlKpb,dmRMlEHSix@gmail.com,TcEDUaWhT,1983,No especificar,"Aleman, Chino",De 1 año a 2 años,Master,No especificar,Orientacion sobre temas legales,Si,Valle Bajo del Lozoya,"Lectura, Baile, Cine",12/07/2021,NaN,NaN
2,3,ZCogidr,pyqWoZEy,InOMyDalbC@gmail.com,YigDCDzGH,1998,Masculino,"Portugues, Ingles",Menos de 6 meses,Doctorado,No especificar,Orientacion laboral,No,Sierra del Rincon,"Viajar, Baile, Lectura",09/02/2021,NaN,NaN
3,4,vdTiFuY,DsyOuIJT,zXBFBgHyDk@gmail.com,juZCtMEMk,1968,Femenino,"Frances, Ucraniano",De 6 meses a 1 año,FP (Formacion Profesional),No especificar,Orientacion sobre tramites,Si,Valle Medio del Lozoya,"Deportes, Pintura, Baile",24/08/2022,NaN,NaN
4,5,sRnfLQq,sOAsLqnv,juObUXPbPT@gmail.com,IlpRnxgfA,1982,No especificar,"Ucraniano, Ingles",De 6 meses a 1 año,Sin graduado escolar,Si,Orientacion sobre temas legales,Si,Valle Alto del Lozoya,"Cocina, Juegos, Musica",21/04/2021,NaN,NaN


In [4]:
# Funcion que utiliza el id recibido para crear una lista con los datos del usuario y un string del tipo de apoyo

def lista_datos(x):
    # database_copia = database.drop(['user_name', 'usersurname', 'email', 'password', 'pic', 'country', 'date', 'studies'],axis=1)
    user=database.loc[database["user_id"] == x]
    tipo_apoyo = user.support_type[x-1]
    lista_user=user.values.flatten().tolist()
    return lista_user, tipo_apoyo

# Función que utiliza como argumento el string de tipo de apoyo y filtra para devolver un dataframe

def tipo_apoyo(apoyo):
    if apoyo == 'Orientacion sobre temas legales':
        df_temas_legales = database[(database['support_type']=='Orientacion sobre temas legales')]
        df_temas_legales = df_temas_legales[df_temas_legales['expert']=='Si']

        return df_temas_legales

    elif apoyo == 'Orientacion sobre tramites':
        df_tramites = database[(database['support_type']=='Orientacion sobre tramites')]
        df_tramites = df_tramites[df_tramites['expert']=='Si']

        return df_tramites

    elif apoyo == 'Orientacion laboral':
        df_tramites_laborales = database[(database['support_type']=='Orientacion laboral')]
        df_tramites_laborales = df_tramites_laborales[df_tramites_laborales['expert']=='Si']

        return df_tramites_laborales

    else:
        df_emocional = database[(database['support_type']=='Emocional')]
        df_emocional = df_emocional[df_emocional['expert']=='Si']

        return df_emocional

# Función que toma como argumentos la lista de datos del usuario y el dataframe de expertos segun el tipo de apoyo y devuelve un df que va para el modelo

def filtro_idioma(datos_usuario,df_support):
    if ',' in datos_usuario[df_support.columns.get_loc('mother_tongue')]:
        datos_usuario[df_support.columns.get_loc('mother_tongue')] = [datos_usuario[df_support.columns.get_loc('mother_tongue')]]
        div_idioma = datos_usuario[df_support.columns.get_loc('mother_tongue')][0].split(', ')
        df_model = df_support[(df_support.mother_tongue.str.contains(div_idioma[0])) | (df_support.mother_tongue.str.contains(div_idioma[1]))]

    else:
        df_model = df_support[(df_support.mother_tongue.str.contains(datos_usuario[df_support.columns.get_loc('mother_tongue')]))]

    return df_model


# Función que ejecuta todas las funciones anteriores para poder realizarlo en una sola vez

def automatizacion(id):
    
    datos_usuario, apoyo = lista_datos(id)

    df_support = tipo_apoyo(apoyo=apoyo)

    df_salida = filtro_idioma(datos_usuario, df_support)
    return df_salida


In [7]:
id = 6
para_modelo = automatizacion(id=id)
para_modelo

,user_id,user_name,usersurname,email,password,year_birth,gender,mother_tongue,years_in,studies,working,support_type,expert,area,about_me,date,country,pic
1,2,vraVenl,pQgDlKpb,dmRMlEHSix@gmail.com,TcEDUaWhT,1983,No especificar,"Aleman, Chino",De 1 año a 2 años,Master,No especificar,Orientacion sobre temas legales,Si,Valle Bajo del Lozoya,"Lectura, Baile, Cine",12/07/2021,NaN,NaN
17,18,VzutOKn,KqzUpHIe,BkWwwyhuRp@gmail.com,JyFqJtRYI,2000,Masculino,"Chino, Español",Más de 2 años,Doctorado,Si,Orientacion sobre temas legales,Si,Valle Alto del Lozoya,"Baile, Naturaleza, Juegos",27/04/2022,NaN,NaN
30,31,mNCoTCn,gAYwdwRu,QlOFQwoVkj@gmail.com,yTCWUzHWS,1992,Femenino,Aleman,Menos de 6 meses,Master,No,Orientacion sobre temas legales,Si,Sierra de la Cabrera,"Naturaleza, Viajar, Fotografia",22/08/2022,NaN,NaN
39,40,HwPfiKy,orWwVTvM,OBEuYrBelW@gmail.com,EosQWcAjP,1968,Femenino,"Aleman, Ingles",De 1 año a 2 años,Sin graduado escolar,Si,Orientacion sobre temas legales,Si,Valle Bajo del Lozoya,"Musica, Lectura, Baile",12/04/2021,NaN,NaN
40,41,KYGfalI,wxakjEzM,tHsVhxArKu@gmail.com,LLTkSXgOz,1990,Masculino,Aleman,Más de 2 años,Con graduado escolar,Si,Orientacion sobre temas legales,Si,Valle Alto del Lozoya,"Lectura, Viajar, Baile",13/03/2021,NaN,NaN
80,81,IWIanKK,VxXHRxrs,GieynuoJcn@gmail.com,ujTMrGOqq,1976,No especificar,"Español, Aleman",Menos de 6 meses,Sin graduado escolar,No,Orientacion sobre temas legales,Si,Valle Medio del Lozoya,"Juegos, Baile, Pintura",18/06/2022,NaN,NaN
112,113,ztfnbpO,jaewnScG,RbMSZRYalt@gmail.com,kUbRKVFsD,1981,Femenino,"Aleman, Español",De 6 meses a 1 año,Con graduado escolar,No especificar,Orientacion sobre temas legales,Si,Sierra del Rincon,"Pintura, Baile, Lectura",02/06/2021,NaN,NaN
116,117,BWjeOsb,kMASKCCD,wnEbHcIiHA@gmail.com,RBNaModIc,1998,Masculino,"Portugues, Chino",De 1 año a 2 años,Doctorado,Si,Orientacion sobre temas legales,Si,Valle del Jarama,"Viajar, Fotografia, Pintura",10/06/2022,NaN,NaN
185,186,UYfWBCa,JleAbuad,NxRssfAoWl@gmail.com,vsdvVMFQN,1985,No especificar,Chino,Más de 2 años,Doctorado,No,Orientacion sobre temas legales,Si,Sierra de la Cabrera,"Cine, Viajar, Fotografia",29/08/2021,NaN,NaN
217,218,QwWmCul,idyoKiNk,CjDVXdBIIi@gmail.com,BqZOEFqpK,1987,Masculino,"Portugues, Chino",Más de 2 años,Master,Si,Orientacion sobre temas legales,Si,Valle Medio del Lozoya,"Juegos, Fotografia, Naturaleza",09/10/2022,NaN,NaN


,user_id,user_name,usersurname,email,password,year_birth,gender,mother_tongue,years_in,studies,working,support_type,expert,area,about_me,date,country,pic
4,5,sRnfLQq,sOAsLqnv,juObUXPbPT@gmail.com,IlpRnxgfA,1982,No especificar,"Ucraniano, Ingles",De 6 meses a 1 año,Sin graduado escolar,Si,Orientacion sobre temas legales,Si,Valle Alto del Lozoya,"Cocina, Juegos, Musica",21/04/2021,NaN,NaN
39,40,HwPfiKy,orWwVTvM,OBEuYrBelW@gmail.com,EosQWcAjP,1968,Femenino,"Aleman, Ingles",De 1 año a 2 años,Sin graduado escolar,Si,Orientacion sobre temas legales,Si,Valle Bajo del Lozoya,"Musica, Lectura, Baile",12/04/2021,NaN,NaN
52,53,rkKDbbh,CJZplosA,XaVaytmKDJ@gmail.com,NHqAklMay,1980,Femenino,"Frances, Ingles",De 1 año a 2 años,Master,No,Orientacion sobre temas legales,Si,Sierra del Rincon,"Juegos, Musica, Baile",27/02/2021,NaN,NaN
71,72,EkmWQNv,FeHIGduz,GDnNNhghLy@gmail.com,pcOSfXYiR,1977,Femenino,"Ingles, Rumano",De 1 año a 2 años,Sin graduado escolar,No,Orientacion sobre temas legales,Si,Valle del Jarama,"Juegos, Pintura, Cocina",18/10/2022,NaN,NaN
88,89,rjOHASc,gBRJCbmS,DesIqkGqda@gmail.com,ngUCKABVS,1988,No especificar,"Ingles, Frances",Más de 2 años,Grado universitario,No,Orientacion sobre temas legales,Si,Sierra del Rincon,"Baile, Cine, Cocina",11/02/2021,NaN,NaN
183,184,ndwDnoS,tCYtdRHt,SdyLTOISSa@gmail.com,bLVtrhCaU,1966,Femenino,Ingles,Menos de 6 meses,Con graduado escolar,No,Orientacion sobre temas legales,Si,Valle del Jarama,"Pintura, Viajar, Musica",22/10/2021,NaN,NaN
194,195,CZchBKo,IEQVSPfT,jmwFrWPgsT@gmail.com,hzZPXxmvv,2000,Masculino,"Ingles, Arabe",De 1 año a 2 años,Grado universitario,Si,Orientacion sobre temas legales,Si,Valle Alto del Lozoya,"Viajar, Fotografia, Baile",26/11/2022,NaN,NaN
274,275,gwUEQFJ,tmurcTGN,DwzkpOWZQU@gmail.com,qyuAmFOHT,1988,Masculino,Ucraniano,De 1 año a 2 años,Grado universitario,Si,Orientacion sobre temas legales,Si,Sierra de la Cabrera,"Cocina, Musica, Deportes",21/09/2021,NaN,NaN
345,346,lpVgxrP,ueNsHwVP,RgzOQLvyYE@gmail.com,tYZsxfVDQ,1983,Femenino,"Ucraniano, Aleman",Menos de 6 meses,Master,No especificar,Orientacion sobre temas legales,Si,Sierra del Rincon,"Lectura, Deportes, Naturaleza",04/12/2021,NaN,NaN
380,381,cffkLbn,pljDTfUP,oszYuONgDU@gmail.com,ekcjXGnTu,1965,Femenino,"Aleman, Ucraniano",De 1 año a 2 años,Master,No especificar,Orientacion sobre temas legales,Si,Sierra de la Cabrera,"Lectura, Viajar, Deportes",05/12/2022,NaN,NaN
